# 全量微调 中文预训练模型

## 2.1 环境设置

In [1]:
import platform
MODEL_PATH = "/opt/Data/ModelWeight/FlagAlpha/Atom-7B-Chat"

## 2.2 模型权重加载

In [2]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

model = AutoModelForCausalLM.from_pretrained(MODEL_PATH,
                                             device_map="cuda:0",
                                             torch_dtype=torch.float16,
                                             load_in_8bit=False) # use_flash_attention_2=True trust_remote_code=True
model =model.eval()

tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH, use_fast=False)
tokenizer.pad_token = tokenizer.eos_token

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Some weights of LlamaForCausalLM were not initialized from the model checkpoint at /opt/Data/ModelWeight/FlagAlpha/Atom-7B-Chat and are newly initialized: ['model.layers.19.self_attn.rotary_emb.inv_freq', 'model.layers.29.self_attn.rotary_emb.inv_freq', 'model.layers.20.self_attn.rotary_emb.inv_freq', 'model.layers.4.self_attn.rotary_emb.inv_freq', 'model.layers.9.self_attn.rotary_emb.inv_freq', 'model.layers.21.self_attn.rotary_emb.inv_freq', 'model.layers.10.self_attn.rotary_emb.inv_freq', 'model.layers.16.self_attn.rotary_emb.inv_freq', 'model.layers.3.self_attn.rotary_emb.inv_freq', 'model.layers.11.self_attn.rotary_emb.inv_freq', 'model.layers.5.self_attn.rotary_emb.inv_freq', 'model.layers.22.self_attn.rotary_emb.inv_freq', 'model.layers.15.self_attn.rotary_emb.inv_freq', 'model.layers.8.self_attn.rotary_emb.inv_freq', 'model.layers.14.self_attn.rotary_emb.inv_freq', 'model.layers.23.self_attn.rotary_emb.inv_freq', 'model.layers.18.self_attn.rotary_emb.inv_freq', 'model.layers.25

## 2.3 生成调用

格式上有要求:

```bash
<s>Human: 你好，介绍一下南京\n</s><s>Assistant: 
```

In [5]:
prompt = ['<s>Human: 你好，介绍一下南京\n</s><s>Assistant: ']

# prompt = ['<s>Human: 你好啊\n</s><s>Assistant: ']

input_ids = tokenizer(prompt, return_tensors="pt", add_special_tokens=False)

generate_input = {
    "input_ids":input_ids.input_ids.to('cuda'),
    "max_new_tokens":4096,
    "top_k":50,
    "top_p":0.95,
    "temperature":1.0,
    # "repetition_penalty":1.3,
    "eos_token_id":tokenizer.eos_token_id,
    "bos_token_id":tokenizer.bos_token_id,
    "pad_token_id":tokenizer.pad_token_id
}

generate_ids  = model.generate(**generate_input)

text = tokenizer.decode(generate_ids[0])
print(text)

<s> Human: 你好，介绍一下南京
</s><s> Assistant: 嗨，南京是一个历史悠久的城市，位于中国东部，长江下游。南京是中国四大古都之一，也是中国重要的历史文化名城。南京有着丰富的历史文化遗产，如南京夫子庙、南京中山陵、南京总统府等。南京也是中国重要的经济中心之一，有着发达的制造业和物流业。南京还是中国重要的交通枢纽之一，有着发达的铁路、公路和航空运输网络。总的来说，南京是一个历史悠久、文化底蕴深厚、经济实力雄厚、交通便利的城市。
</s>


In [18]:
def tripStr(text):
    assist = "Assistant: "
    if assist in text:
        pos = text.index(assist) + len(assist)
        data = text[pos:len(text)]
        data=data.replace("</s>","")
        return data

    return text

data = tripStr(text)
print(data)

嗨，南京是一个历史悠久的城市，位于中国东部，长江下游。南京是中国四大古都之一，也是中国重要的历史文化名城。南京有着丰富的历史文化遗产，如南京夫子庙、南京中山陵、南京总统府等。南京也是中国重要的经济中心之一，有着发达的制造业和物流业。南京还是中国重要的交通枢纽之一，有着发达的铁路、公路和航空运输网络。总的来说，南京是一个历史悠久、文化底蕴深厚、经济实力雄厚、交通便利的城市。

